In [1]:
import numpy as np
import pandas as pd
import json
import math
import pickle
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import os
from datetime import datetime

In [2]:
original_data = pd.read_csv('goodreads_interactions.csv')

#CSV columns:
#user_id | book_id | is_read | rating | is_reviewed

In [ ]:
original_data.head()

In [4]:
print(f"Original dataset:")
print(f"Interactions: {len(original_data):,}")
print(f"Users: {original_data['user_id'].nunique():,}")
print(f"Books: {original_data['book_id'].nunique():,}")

Original dataset:
Interactions: 123,731,803
Users: 254,205
Books: 2,144,808


In [5]:
# Count how many times each book was interacted with
book_interactions = original_data.groupby('book_id').size().reset_index(name='n_interactions')

print(f"\nBasic Statistics:")
print(f"Min interactions: {book_interactions['n_interactions'].min()}")
print(f"Max interactions: {book_interactions['n_interactions'].max():,}")
print(f"Mean interactions: {book_interactions['n_interactions'].mean():.2f}")
print(f"Median interactions: {book_interactions['n_interactions'].median():.1f}")
print(f"Std deviation: {book_interactions['n_interactions'].std():.2f}")


Basic Statistics:
Min interactions: 1
Max interactions: 129,935
Mean interactions: 57.69
Median interactions: 6.0
Std deviation: 614.02


In [8]:
print(f"Percentiles:")
percentiles = [10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    val = book_interactions['n_interactions'].quantile(p/100)
    print(f"{p}th percentile: {val:.0f} interactions")

Percentiles:
10th percentile: 1 interactions
25th percentile: 2 interactions
50th percentile: 6 interactions
75th percentile: 22 interactions
90th percentile: 76 interactions
95th percentile: 162 interactions
99th percentile: 780 interactions


In [9]:
print("Rating distribution:")
rating_dist = original_data['rating'].value_counts().sort_index()
for rating, count in rating_dist.items():
    print(f"{rating:.1f} stars: {count:,} ({100*count/len(original_data):.1f}%)")

Rating distribution:
0.0 stars: 68,207,936 (55.1%)
1.0 stars: 1,102,066 (0.9%)
2.0 stars: 3,349,217 (2.7%)
3.0 stars: 12,633,819 (10.2%)
4.0 stars: 20,111,311 (16.3%)
5.0 stars: 18,327,453 (14.8%)


In [10]:
# Filter out zero/missing ratings
data = original_data[original_data['rating'] > 0].copy()
print(f"After filtering: {len(data):,} interactions")

After filtering: 55,523,866 interactions


In [12]:
# FILTER BY MINIMUM INTERACTIONS

In [11]:
user_counts = data['user_id'].value_counts()
book_counts = data['book_id'].value_counts()
print(f"Users: {len(user_counts):,}")
print(f"Books: {len(book_counts):,}")

Users: 254,205
Books: 2,144,808


In [13]:
valid_users = set(user_counts[user_counts >= 10].index)
valid_books = set(book_counts[book_counts >= 50].index)

print(f"Valid users (>=10 interactions): {len(valid_users):,}")
print(f"Valid books (>=50 interactions): {len(valid_books):,}")

Valid users (>=10 interactions): 254,196
Valid books (>=50 interactions): 306,020


In [14]:
mask = data['user_id'].isin(valid_users) & data['book_id'].isin(valid_books)
data = data[mask].copy()
print(f"Filtered: {len(data):,} interactions")
print(f"Books: {data['book_id'].nunique():,}")
print(f"Users: {data['user_id'].nunique():,}")

Filtered: 48,005,457 interactions
Books: 303,318
Users: 252,841


In [15]:
print(f"FINAL SAMPLED DATASET:")
print(f"Interactions: {len(data):,}")
print(f"Books: {data['book_id'].nunique():,}")
print(f"Users: {data['user_id'].nunique():,}")
print(f"Sparsity: {100 * (1 - len(data) / (data['user_id'].nunique() * data['book_id'].nunique())):.2f}%")
print(f"Avg interactions/book: {len(data) / data['book_id'].nunique():.1f}")
print(f"Avg interactions/user: {len(data) / data['user_id'].nunique():.1f}")

FINAL SAMPLED DATASET:
Interactions: 48,005,457
Books: 303,318
Users: 252,841
Sparsity: 99.94%
Avg interactions/book: 158.3
Avg interactions/user: 189.9


In [16]:
print(f"Rating statistics:")
print(f"Min: {data['rating'].min():.1f}")
print(f"Max: {data['rating'].max():.1f}")
print(f"Mean: {data['rating'].mean():.2f}")
print(f"Median: {data['rating'].median():.1f}")

Rating statistics:
Min: 1.0
Max: 5.0
Mean: 3.94
Median: 4.0


In [17]:
folder = r"D:\GoodreadsData_NEW_2"
os.makedirs(folder, exist_ok=True)
data.to_csv(os.path.join(folder, 'sampled_data.csv'), index=False)
print(f"\n✓ Sampled data saved to: {folder}")


✓ Sampled data saved to: D:\GoodreadsData_NEW_2


In [3]:
import pandas as pd
import os

folder = r"D:\GoodreadsData_NEW_2"
file_path = os.path.join(folder, "sampled_data.csv")  

data = pd.read_csv(file_path)
print("Loaded rows:", len(data))

Loaded rows: 48005457


In [5]:
target_size = 1000000

if len(data) > target_size:
    print(f"Dataset too large ({len(data):,}), sampling to {target_size:,}...")
    data = data.sample(n=target_size, random_state=42)
    
    # Re-filter after sampling
    book_counts = data['book_id'].value_counts()
    valid_books = book_counts[book_counts >= 20].index
    data = data[data['book_id'].isin(valid_books)].copy()
    
    user_counts = data['user_id'].value_counts()
    valid_users = user_counts[user_counts >= 5].index
    data = data[data['user_id'].isin(valid_users)].copy()

In [6]:
# CREATE INDEX MAPPINGS

users = sorted(data['user_id'].unique())
books = sorted(data['book_id'].unique())

user_to_idx = {user: idx for idx, user in enumerate(users)}
book_to_idx = {book: idx for idx, book in enumerate(books)}
idx_to_user = {idx: user for user, idx in user_to_idx.items()}
idx_to_book = {idx: book for book, idx in book_to_idx.items()}

print(f"Users: {len(users):,}")
print(f"Books: {len(books):,}")

# Create helper dictionaries
user_to_books = data.groupby('user_id')['book_id'].apply(list).to_dict()
book_to_users = data.groupby('book_id')['user_id'].apply(list).to_dict()
user_to_ratings = data.groupby('user_id')['rating'].apply(list).to_dict()

# Create user_book_to_rating mapping
user_book_to_rating = {}
for _, row in data.iterrows():
    user_book_to_rating[(row['user_id'], row['book_id'])] = row['rating']

print(f"Created {len(user_book_to_rating):,} user-book rating pairs")

Users: 31,401
Books: 7,303
Created 223,922 user-book rating pairs


In [7]:
user_book_to_rating

{(19312.0, 8426.0): 4.0,
 (55724.0, 3603.0): 4.0,
 (159874.0, 19070.0): 5.0,
 (224755.0, 1046.0): 5.0,
 (251951.0, 40206.0): 3.0,
 (219172.0, 8037.0): 2.0,
 (17342.0, 6806.0): 4.0,
 (93741.0, 12906.0): 4.0,
 (75171.0, 10633.0): 5.0,
 (192237.0, 24429.0): 4.0,
 (132082.0, 27662.0): 5.0,
 (34612.0, 21089.0): 5.0,
 (119863.0, 35048.0): 1.0,
 (227350.0, 742.0): 3.0,
 (30362.0, 11556.0): 5.0,
 (185842.0, 19473.0): 4.0,
 (243837.0, 14872.0): 3.0,
 (113119.0, 28541.0): 4.0,
 (53240.0, 13796.0): 3.0,
 (123145.0, 26809.0): 4.0,
 (110496.0, 1000.0): 4.0,
 (187093.0, 12801.0): 4.0,
 (95938.0, 2157.0): 5.0,
 (62022.0, 6730.0): 5.0,
 (241990.0, 588.0): 3.0,
 (150311.0, 1606.0): 5.0,
 (11649.0, 2860.0): 3.0,
 (183964.0, 48793.0): 3.0,
 (244207.0, 25895.0): 3.0,
 (112479.0, 6182.0): 4.0,
 (228250.0, 8424.0): 2.0,
 (204342.0, 15090.0): 5.0,
 (84710.0, 7280.0): 4.0,
 (80145.0, 21401.0): 5.0,
 (37825.0, 8775.0): 4.0,
 (64763.0, 12560.0): 5.0,
 (21405.0, 13399.0): 5.0,
 (178184.0, 968.0): 5.0,
 (134118.0

In [8]:
user_to_books

{5: [7043, 2787, 407, 5365, 6616, 6460],
 8: [461, 855, 994, 7279, 7170, 828],
 14: [12572, 11878, 6379, 10327, 6700, 9399, 9024, 10410, 12177],
 16: [3777, 92, 998, 689, 12966],
 20: [14238, 12582, 14237, 468, 14066, 8399],
 28: [6700, 16043, 5691, 12221, 1012, 13577, 9482, 11505, 12645, 5921, 9399],
 30: [17183, 8411, 43, 17061, 716, 17102, 8462, 17052],
 32: [5879, 6960, 16269, 833, 772, 17588, 446, 17525, 17186],
 41: [6683, 1387, 5238, 19468, 19566, 19520],
 66: [21369, 7050, 12948, 17148, 1611, 19859, 16141, 7973],
 78: [14677, 1591, 23636, 7049, 23164, 5238, 1000],
 100: [1387, 1221, 443, 7993, 66],
 138: [39351, 17322, 13975, 35170, 787],
 142: [6256, 19377, 1471, 40105, 4465, 6868, 1233, 7014, 19588],
 161: [6807, 5681, 32601, 16103, 32605],
 190: [13372, 1368, 1212, 7220, 8043],
 195: [2403, 7008, 1204, 7467, 20002, 7469, 7452],
 214: [18286, 941, 846, 7000, 1117, 56519],
 219: [7000, 57723, 37092, 26168, 57413],
 226: [704,
  13313,
  12767,
  28540,
  58868,
  1611,
  15504

In [11]:
import pickle
import os

folder = r"D:\GoodreadsData\mappings"
os.makedirs(folder, exist_ok=True)

mappings = {
    "users.pkl": users,
    "books.pkl": books,
    "user_to_idx.pkl": user_to_idx,
    "book_to_idx.pkl": book_to_idx,
    "idx_to_user.pkl": idx_to_user,
    "idx_to_book.pkl": idx_to_book,
    "user_to_books.pkl": user_to_books,
    "book_to_users.pkl": book_to_users,
    "user_to_ratings.pkl": user_to_ratings,
    "user_book_to_rating.pkl": user_book_to_rating,
}

for filename, obj in mappings.items():
    with open(os.path.join(folder, filename), "wb") as f:
        pickle.dump(obj, f)

print("All mappings saved!")

All mappings saved!


In [12]:
# CREATE SPARSE RATING MATRIX
row_indices = [user_to_idx[uid] for uid in data['user_id']]
col_indices = [book_to_idx[bid] for bid in data['book_id']]
ratings = data['rating'].values

rating_matrix = csr_matrix(
    (ratings, (row_indices, col_indices)),
    shape=(len(users), len(books)),
    dtype=np.float32
)

print(f"Matrix shape: {rating_matrix.shape}")
print(f"Non-zero entries: {rating_matrix.nnz:,}")
print(f"Memory: ~{rating_matrix.data.nbytes / (1024**3):.2f} GB")

Matrix shape: (252841, 303318)
Non-zero entries: 223,922
Memory: ~0.00 GB


In [13]:
# COMPUTE USER MEANS AND CENTER RATINGS

# Compute user means
user_rating_counts = np.diff(rating_matrix.indptr)
user_rating_sums = np.array(rating_matrix.sum(axis=1)).flatten()

user_means = np.divide(
    user_rating_sums, 
    user_rating_counts, 
    out=np.zeros_like(user_rating_sums), 
    where=user_rating_counts > 0
)

print(f"Mean user rating: {user_means[user_means > 0].mean():.3f}")
print(f"Min user mean: {user_means[user_means > 0].min():.3f}")
print(f"Max user mean: {user_means.max():.3f}")

# Center ratings (subtract user means)
rating_matrix_centered = rating_matrix.copy().astype(np.float32)

for user_idx in range(rating_matrix_centered.shape[0]):
    start = rating_matrix_centered.indptr[user_idx]
    end = rating_matrix_centered.indptr[user_idx + 1]
    
    if end > start and user_means[user_idx] > 0:
        rating_matrix_centered.data[start:end] -= user_means[user_idx]

# Clean NaN values
nan_count = np.isnan(rating_matrix_centered.data).sum()
if nan_count > 0:
    print(f"Cleaning {nan_count:,} NaN values...")
    rating_matrix_centered.data = np.nan_to_num(rating_matrix_centered.data, nan=0.0)

rating_matrix_centered.eliminate_zeros()

print(f"Ratings centered!")
print(f"Non-zero entries after centering: {rating_matrix_centered.nnz:,}")

Mean user rating: 3.933
Min user mean: 1.000
Max user mean: 5.000
Ratings centered!
Non-zero entries after centering: 205,950


In [14]:
# Computing item-item similarity matrix

start_time = datetime.now()

# Compute adjusted cosine similarity
item_similarity_sparse = cosine_similarity(
    rating_matrix_centered.T, 
    dense_output=False
)

elapsed = datetime.now() - start_time
print(f"Similarity computed in {elapsed}")
print(f"Shape: {item_similarity_sparse.shape}")
print(f"Non-zero similarities: {item_similarity_sparse.nnz:,}")
print(f"Memory: ~{item_similarity_sparse.data.nbytes / (1024**3):.2f} GB")

Similarity computed in 0:00:00.117666
Shape: (303318, 303318)
Non-zero similarities: 1,332,441
Memory: ~0.00 GB


In [16]:
# # FILTER BY MINIMUM COMMON USERS

# # Compute co-occurrence matrix
# rating_binary = rating_matrix.copy()
# rating_binary.data = np.ones_like(rating_binary.data)
# cooccurrence = (rating_binary.T @ rating_binary).toarray()

# # Convert to dense for filtering
# item_similarity = item_similarity_sparse.toarray()

# # Apply minimum common users filter
# min_common = 10
# print(f"Applying filter (min_common={min_common})...")
# item_similarity[cooccurrence < min_common] = 0
# np.fill_diagonal(item_similarity, 1.0)

# # Convert back to sparse
# item_similarity_sparse = csr_matrix(item_similarity)
# del item_similarity  # Free memory

# print(f"Filtered!")
# print(f"Non-zero similarities: {item_similarity_sparse.nnz:,}")
# print(f"Average similarities per book: {item_similarity_sparse.nnz / len(books):.1f}")

  Computing co-occurrence matrix...


MemoryError: Unable to allocate 343. GiB for an array with shape (303318, 303318) and data type float32

In [17]:
from collections import defaultdict
import numpy as np

# Computing sparse co-occurrence (user-wise)

cooccurrence = defaultdict(int)

for user_idx in range(rating_matrix.shape[0]):
    start = rating_matrix.indptr[user_idx]
    end = rating_matrix.indptr[user_idx + 1]
    
    books = rating_matrix.indices[start:end]
    books = np.unique(books)

    # For each pair of books the user interacted with
    for i in range(len(books)):
        for j in range(i + 1, len(books)):
            b1, b2 = books[i], books[j]
            cooccurrence[(b1, b2)] += 1

print(f"Co-occurrence pairs: {len(cooccurrence):,}")

Co-occurrence pairs: 723,984


In [18]:
min_common = 5

filtered_sims = item_similarity_sparse.copy().tolil()

count = 0
for (b1, b2), c in cooccurrence.items():
    if c < min_common:
        filtered_sims[b1, b2] = 0
        filtered_sims[b2, b1] = 0
    count += 1

filtered_sims = filtered_sims.tocsr()

In [19]:
# Computing book means

book_rating_counts = np.diff(rating_matrix.tocsc().indptr)
book_rating_sums = np.array(rating_matrix.sum(axis=0)).flatten()
book_means = np.divide(
    book_rating_sums, 
    book_rating_counts,
    out=np.zeros_like(book_rating_sums),
    where=book_rating_counts > 0
)

print(f"Mean book rating: {book_means[book_means > 0].mean():.3f}")
print(f"Min book rating: {book_means[book_means > 0].min():.3f}")
print(f"Max book rating: {book_means.max():.3f}")

# Compute global mean
global_mean = data['rating'].mean()
print(f"Global mean rating: {global_mean:.3f}")

Mean book rating: 3.891
Min book rating: 1.000
Max book rating: 5.000
Global mean rating: 3.917


In [20]:
# Saving model

save_data = {
    # Core matrices
    'rating_matrix': rating_matrix,
    'item_similarity': item_similarity_sparse,
    
    # Statistics
    'user_means': user_means,
    'book_means': book_means,
    'global_mean': global_mean,
    
    # Index mappings
    'user_to_idx': user_to_idx,
    'book_to_idx': book_to_idx,
    'idx_to_user': idx_to_user,
    'idx_to_book': idx_to_book,
    
    # Helper dictionaries
    'user_to_books': user_to_books,
    'book_to_users': book_to_users,
    'user_to_ratings': user_to_ratings,
    'user_book_to_rating': user_book_to_rating,
    
    # Parameters
    'min_common': min_common,
    
    # Metadata
    'n_users': len(users),
    'n_books': len(books),
    'n_interactions': len(data),
    'creation_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

model_file = os.path.join(folder, 'book_recommender_model.pkl')
print(f"Saving to: {model_file}")

with open(model_file, 'wb') as f:
    pickle.dump(save_data, f, protocol=pickle.HIGHEST_PROTOCOL)

file_size = os.path.getsize(model_file) / (1024**2)
print(f"Model saved! Size: {file_size:.1f} MB")

Saving to: D:\GoodreadsData\mappings\book_recommender_model.pkl
Model saved! Size: 842.4 MB


In [21]:
# STEP 11: DEFINE PREDICTION FUNCTIONS

def predict_rating(user_id, book_id, k=25):
    """
    Predict rating for a user-book pair using k-nearest neighbors
    
    Parameters:
    -----------
    user_id : int
        User ID
    book_id : int
        Book ID
    k : int
        Number of neighbors to use (default: 25)
        
    Returns:
    --------
    float : Predicted rating (0-5)
    """
    #handle cold start
    if user_id not in user_to_idx:
        if book_id in book_to_idx:
            return book_means[book_to_idx[book_id]]
        return global_mean
    
    if book_id not in book_to_idx:
        return user_means[user_to_idx[user_id]]
    
    user_idx = user_to_idx[user_id]
    book_idx = book_to_idx[book_id]
    
    #get user's rated books
    user_ratings = rating_matrix[user_idx].toarray().flatten()
    rated_mask = user_ratings > 0
    
    if not rated_mask.any():
        return book_means[book_idx]
    
    #get similarities to target book
    similarities = item_similarity_sparse[book_idx].toarray().flatten()
    similarities = similarities * rated_mask
    
    #get top-k neighbors
    neighbor_indices = np.argsort(np.abs(similarities))[-k:]
    neighbor_indices = neighbor_indices[similarities[neighbor_indices] != 0]
    
    if len(neighbor_indices) == 0:
        return book_means[book_idx]
    
    #weighted average prediction
    numerator = 0.0
    denominator = 0.0
    
    for idx in neighbor_indices:
        sim = similarities[idx]
        rating = user_ratings[idx]
        baseline = book_means[idx]
        numerator += sim * (rating - baseline)
        denominator += abs(sim)
    
    if denominator == 0:
        return book_means[book_idx]
    
    prediction = book_means[book_idx] + (numerator / denominator)
    return np.clip(prediction, 0, 5)


def get_recommendations(user_id, n=10, k=25):
    """
    Get top N book recommendations for a user
    
    Parameters:
    -----------
    user_id : int
        User ID
    n : int
        Number of recommendations (default: 10)
    k : int
        Number of neighbors for prediction (default: 25)
        
    Returns:
    --------
    list : List of (book_id, predicted_rating) tuples
    """
    #handle cold start users
    if user_id not in user_to_idx:
        top_books = np.argsort(book_means)[-n:][::-1]
        return [(idx_to_book[idx], book_means[idx]) for idx in top_books]
    
    user_idx = user_to_idx[user_id]
    
    #get rated books
    user_ratings = rating_matrix[user_idx].toarray().flatten()
    rated_books = set(np.where(user_ratings > 0)[0])
    
    #get unrated books
    all_books = set(range(len(book_means)))
    unrated_books = list(all_books - rated_books)
    
    #predict ratings for all unrated books
    predictions = []
    for book_idx in unrated_books:
        book_id = idx_to_book[book_idx]
        pred_rating = predict_rating(user_id, book_id, k=k)
        predictions.append((book_id, pred_rating))
    
    #sort and return top N
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:n]


def get_similar_books(book_id, n=10):
    """
    Get most similar books to a given book
    
    Parameters:
    -----------
    book_id : int
        Book ID
    n : int
        Number of similar books (default: 10)
        
    Returns:
    --------
    list : List of (book_id, similarity) tuples
    """
    if book_id not in book_to_idx:
        return []
    
    book_idx = book_to_idx[book_id]
    similarities = item_similarity_sparse[book_idx].toarray().flatten()
    
    # Get top N (excluding the book itself)
    top_indices = np.argsort(similarities)[-(n+1):-1][::-1]
    
    similar_books = [(idx_to_book[idx], similarities[idx]) 
                    for idx in top_indices if similarities[idx] > 0]
    
    return similar_books

print("Functions defined!")

# ---------------- TEST THE MODEL

print("\nTesting the model...")

#test with a random user who has rated multiple books
test_user = None
for uid in user_to_books:
    if len(user_to_books[uid]) >= 3:
        test_user = uid
        break

if test_user is None:
    test_user = list(user_to_idx.keys())[0]

print(f"\nTest User: {test_user}")

#get user's actual ratings
user_idx = user_to_idx[test_user]
user_ratings = rating_matrix[user_idx].toarray().flatten()
rated_indices = np.where(user_ratings > 0)[0]
rated_books = [(idx_to_book[idx], user_ratings[idx]) for idx in rated_indices]

print(f"\nUser has rated {len(rated_books)} books")
print("Sample ratings:")
for book_id, rating in rated_books[:5]:
    print(f"  Book {book_id}: {rating:.1f} stars")

#test prediction accuracy
if len(rated_books) > 0:
    test_book = rated_books[0][0]
    actual_rating = rated_books[0][1]
    predicted = predict_rating(test_user, test_book, k=25)
    print(f"\nPrediction test (on a book the user actually rated):")
    print(f"  Book ID: {test_book}")
    print(f"  Actual rating: {actual_rating:.1f}")
    print(f"  Predicted rating: {predicted:.2f}")
    print(f"  Error: {abs(predicted - actual_rating):.2f}")

#get recommendations
print(f"\nTop 10 Recommendations for User {test_user}:")
recs = get_recommendations(test_user, n=10, k=25)
for i, (book_id, pred_rating) in enumerate(recs, 1):
    print(f"  {i:2d}. Book {book_id:6d} - Predicted: {pred_rating:.2f} stars")

#test similar books
if len(rated_books) > 0:
    test_book = rated_books[0][0]
    print(f"\nBooks similar to Book {test_book}:")
    similar = get_similar_books(test_book, n=5)
    for i, (book_id, similarity) in enumerate(similar, 1):
        print(f"  {i}. Book {book_id} - Similarity: {similarity:.3f}")

# FINAL SUMMARY
print("MODEL TRAINING COMPLETE!")

print("\nModel Summary:")
print(f"Dataset: {len(data):,} interactions")
print(f"Users: {len(users):,}")
print(f"Books: {len(books):,}")
print(f"Model size: {file_size:.1f} MB")
print(f"Training time: ~{(datetime.now() - start_time).seconds // 60} minutes")
print(f"Saved to: {model_file}")

Functions defined!

Testing the model...

Test User: 0

User has rated 0 books
Sample ratings:

Top 10 Recommendations for User 0:
   1. Book   1582 - Predicted: 5.00 stars
   2. Book  13276 - Predicted: 5.00 stars
   3. Book  21779 - Predicted: 5.00 stars
   4. Book  23178 - Predicted: 5.00 stars
   5. Book  23819 - Predicted: 5.00 stars
   6. Book  30947 - Predicted: 5.00 stars
   7. Book  39455 - Predicted: 5.00 stars
   8. Book  49350 - Predicted: 5.00 stars
   9. Book  57728 - Predicted: 5.00 stars
  10. Book  60440 - Predicted: 5.00 stars
MODEL TRAINING COMPLETE!

Model Summary:
Dataset: 223,922 interactions
Users: 252,841
Books: 0
Model size: 842.4 MB
Training time: ~22 minutes
Saved to: D:\GoodreadsData\mappings\book_recommender_model.pkl


In [22]:
model_path = r"D:\GoodreadsData\mappings\book_recommender_model.pkl"

with open(model_path, 'rb') as f:
    model = pickle.load(f)

In [23]:
rating_matrix = model['rating_matrix']
item_similarity = model['item_similarity']
user_means = model['user_means']
book_means = model['book_means']
global_mean = model['global_mean']
user_to_idx = model['user_to_idx']
book_to_idx = model['book_to_idx']
idx_to_user = model['idx_to_user']
idx_to_book = model['idx_to_book']
min_common = model['min_common']

print(f"Model loaded successfully!")
print(f"Users: {len(user_to_idx):,}")
print(f"Books: {len(book_to_idx):,}")
print(f"Interactions: {rating_matrix.nnz:,}")

✓ Model loaded successfully!
  Users: 252,841
  Books: 303,318
  Interactions: 223,922


In [24]:
data

,user_id,book_id,is_read,rating,is_reviewed
4015787,19312,8426,1.0,4.0,0.0
11512196,55724,3603,1.0,4.0,0.0
31395095,159874,19070,1.0,5.0,0.0
42862672,224755,1046,1.0,5.0,0.0
47596028,251951,40206,1.0,3.0,0.0
...,...,...,...,...,...
36217842,186880,15719,1.0,4.0,0.0
22063264,110461,11078,1.0,2.0,0.0
31929137,162597,7051,1.0,2.0,0.0
11779085,57102,14057,1.0,3.0,0.0


In [25]:
training_percentage = 0.8
training_num = int(training_percentage * len(data))

train_data = data.iloc[:training_num]
test_data = data.iloc[training_num:]

print(f"Training set: {len(train_data):,} interactions")
print(f"Test set: {len(test_data):,} interactions")

#filter test data to only include users and books in the model
test_data_filtered = test_data[
    test_data['user_id'].isin(user_to_idx.keys()) & 
    test_data['book_id'].isin(book_to_idx.keys())
].copy()

print(f"Test set (filtered): {len(test_data_filtered):,} interactions")

#create test dictionary
user_book_to_rating_test = {}
for _, row in test_data_filtered.iterrows():
    user_book_to_rating_test[(row['user_id'], row['book_id'])] = row['rating']

print(f"Test pairs: {len(user_book_to_rating_test):,}")

Training set: 179,137 interactions
Test set: 44,785 interactions
Test set (filtered): 44,785 interactions
Test pairs: 44,785


In [26]:
def predict_rating(user_id, book_id, k=25):
    """
    Predict rating using Item-Based Collaborative Filtering
    
    Parameters:
    -----------
    user_id : int
        User ID
    book_id : int
        Book ID
    k : int
        Number of neighbors to use
        
    Returns:
    --------
    float : Predicted rating (0-5)
    """
    #handle cold start
    if user_id not in user_to_idx:
        if book_id in book_to_idx:
            return book_means[book_to_idx[book_id]]
        return global_mean
    
    if book_id not in book_to_idx:
        return user_means[user_to_idx[user_id]]
    
    user_idx = user_to_idx[user_id]
    book_idx = book_to_idx[book_id]
    
    #get user's rated books
    user_ratings = rating_matrix[user_idx].toarray().flatten()
    rated_mask = user_ratings > 0
    
    if not rated_mask.any():
        return book_means[book_idx]
    
    #get similarities to target book
    similarities = item_similarity[book_idx].toarray().flatten()
    similarities = similarities * rated_mask
    
    #get top-k neighbors
    neighbor_indices = np.argsort(np.abs(similarities))[-k:]
    neighbor_indices = neighbor_indices[similarities[neighbor_indices] != 0]
    
    if len(neighbor_indices) == 0:
        return book_means[book_idx]
    
    #weighted average prediction
    numerator = 0.0
    denominator = 0.0
    
    for idx in neighbor_indices:
        sim = similarities[idx]
        rating = user_ratings[idx]
        baseline = book_means[idx]
        numerator += sim * (rating - baseline)
        denominator += abs(sim)
    
    if denominator == 0:
        return book_means[book_idx]
    
    prediction = book_means[book_idx] + (numerator / denominator)
    return np.clip(prediction, 0, 5)

print("Prediction function defined!")


Prediction function defined!


In [27]:
def evaluate_rating_accuracy(test_dict, k_neighbors=25, sample_size=None):
    """
    Evaluate RMSE and MAE on test set
    
    Parameters:
    -----------
    test_dict : dict
        Dictionary of (user_id, book_id) -> rating
    k_neighbors : int
        Number of neighbors for prediction
    sample_size : int or None
        If provided, evaluate on random sample
        
    Returns:
    --------
    tuple : (rmse, mae, coverage)
    """
    predictions = []
    actuals = []
    
    test_items = list(test_dict.items())
    
    if sample_size and sample_size < len(test_items):
        test_items = np.random.choice(len(test_items), sample_size, replace=False)
        test_items = [(list(test_dict.keys())[i], list(test_dict.values())[i]) 
                     for i in test_items]
    else:
        test_items = [(k, v) for k, v in test_dict.items()]
    
    print(f"  Evaluating on {len(test_items):,} test cases...")
    
    for i, ((user_id, book_id), actual_rating) in enumerate(test_items):
        if (i + 1) % 1000 == 0:
            print(f"    Processed {i+1:,} / {len(test_items):,} predictions...")
        
        predicted_rating = predict_rating(user_id, book_id, k=k_neighbors)
        predictions.append(predicted_rating)
        actuals.append(actual_rating)
    
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    #calculate metrics
    rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
    mae = np.mean(np.abs(predictions - actuals))
    coverage = 1.0  # Item-CF typically has full coverage
    
    return rmse, mae, coverage

# ealuate with different K values
print("Testing different K values:")
k_values = [10, 25, 50]

rating_results = {}
for k in k_values:
    print(f"K = {k}:")
    rmse, mae, coverage = evaluate_rating_accuracy(
        user_book_to_rating_test, 
        k_neighbors=k,
        sample_size=5000  
    )
    rating_results[k] = {'RMSE': rmse, 'MAE': mae, 'Coverage': coverage}
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Coverage: {coverage:.2%}")


Testing different K values:

  K = 10:
  Evaluating on 5,000 test cases...
    Processed 1,000 / 5,000 predictions...
    Processed 2,000 / 5,000 predictions...
    Processed 3,000 / 5,000 predictions...
    Processed 4,000 / 5,000 predictions...
    Processed 5,000 / 5,000 predictions...
    RMSE: 0.1220
    MAE: 0.0609
    Coverage: 100.00%

  K = 25:
  Evaluating on 5,000 test cases...
    Processed 1,000 / 5,000 predictions...
    Processed 2,000 / 5,000 predictions...
    Processed 3,000 / 5,000 predictions...
    Processed 4,000 / 5,000 predictions...
    Processed 5,000 / 5,000 predictions...
    RMSE: 0.1277
    MAE: 0.0627
    Coverage: 100.00%

  K = 50:
  Evaluating on 5,000 test cases...
    Processed 1,000 / 5,000 predictions...
    Processed 2,000 / 5,000 predictions...
    Processed 3,000 / 5,000 predictions...
    Processed 4,000 / 5,000 predictions...
    Processed 5,000 / 5,000 predictions...
    RMSE: 0.1255
    MAE: 0.0618
    Coverage: 100.00%


========================================================== ENDS HERE ================================================

In [ ]:
def evaluate_ranking_metrics(test_dict, k_neighbors=25, k_items=10, 
                            relevance_threshold=4.0, max_users=100):
    """
    Evaluate Precision@K, Recall@K, and F1@K

    Parameters:
    -----------
    test_dict : dict
        Dictionary of (user_id, book_id) -> rating
    k_neighbors : int
        Number of neighbors for prediction
    k_items : int
        Number of top items to recommend
    relevance_threshold : float
        Minimum rating to consider relevant
    max_users : int
        Maximum number of users to evaluate (for speed)

    Returns:
    --------
    tuple : (avg_precision, avg_recall, avg_f1)
    """
    # Get all books in test set
    all_books = {book for (user, book), rating in test_dict.items()}

    # Get relevant books for each user (ratings >= threshold)
    user_to_books_relevant = {}
    for (user, book), rating in test_dict.items():
        if rating >= relevance_threshold:
            if user not in user_to_books_relevant:
                user_to_books_relevant[user] = set()
            user_to_books_relevant[user].add(book)

    print(f"  Users with relevant items: {len(user_to_books_relevant)}")

    # Sample users if too many
    users_to_evaluate = list(user_to_books_relevant.keys())
    if len(users_to_evaluate) > max_users:
        users_to_evaluate = np.random.choice(users_to_evaluate, max_users, replace=False)
        print(f"  Sampling {max_users} users for evaluation...")

    metrics_list = []

    for idx, user in enumerate(users_to_evaluate):
        if (idx + 1) % 10 == 0:
            print(f"    Evaluated {idx+1} / {len(users_to_evaluate)} users...")

        ground_truth = user_to_books_relevant[user]

        # Get user's training books (already rated)
        user_idx = user_to_idx[user]
        user_ratings = rating_matrix[user_idx].toarray().flatten()
        training_books = set([idx_to_book[i] for i in np.where(user_ratings > 0)[0]])

        # Get candidate books (not in training)
        candidate_books = list(all_books - training_books)

        # Predict ratings for all candidates
        predictions = []
        for book in candidate_books:
            pred_rating = predict_rating(user, book, k=k_neighbors)
            predictions.append((book, pred_rating))

        # Sort by predicted rating and get top-K
        predictions_sorted = sorted(predictions, key=lambda x: x[1], reverse=True)
        top_k_books = [book for book, score in predictions_sorted[:k_items]]

        # Calculate metrics
        num_relevant_in_top_k = len(set(top_k_books).intersection(ground_truth))

        precision = num_relevant_in_top_k / k_items if k_items > 0 else 0
        recall = num_relevant_in_top_k / len(ground_truth) if len(ground_truth) > 0 else 0

        if (precision + recall) == 0:
            f1 = 0
        else:
            f1 = 2 * precision * recall / (precision + recall)

        metrics_list.append((precision, recall, f1))

    # Calculate averages
    avg_precision = np.mean([m[0] for m in metrics_list])
    avg_recall = np.mean([m[1] for m in metrics_list])
    avg_f1 = np.mean([m[2] for m in metrics_list])

    return avg_precision, avg_recall, avg_f1
# Evaluate ranking metrics
print("\nEvaluating ranking metrics (this may take a while)...")
RELEVANCE_THRESHOLD = 4.0
k_items = 10
k_neighbors = 25
avg_precision, avg_recall, avg_f1 = evaluate_ranking_metrics(
    user_book_to_rating_test,
    k_neighbors=k_neighbors,
    k_items=k_items,
    relevance_threshold=RELEVANCE_THRESHOLD,
    max_users=100  # Evaluate on 100 users for speed
)
print(f"\n  Results for K={k_neighbors}, Top-{k_items} recommendations:")
print(f"    Precision@{k_items}: {avg_precision:.4f}")
print(f"    Recall@{k_items}: {avg_recall:.4f}")
print(f"    F1@{k_items}: {avg_f1:.4f}")


Evaluating ranking metrics (this may take a while)...
  Users with relevant items: 19691
  Sampling 100 users for evaluation...
    Evaluated 10 / 100 users...
    Evaluated 20 / 100 users...
    Evaluated 30 / 100 users...
    Evaluated 40 / 100 users...
    Evaluated 50 / 100 users...
    Evaluated 60 / 100 users...


In [33]:
# def evaluate_ranking_metrics_fast(test_dict, k_neighbors=25, k_items=10, 
#                                   relevance_threshold=4.0, max_users=50):
#     """
#     Fast evaluation of Precision@K, Recall@K, and F1@K
#     """
#     # Get relevant books for each user (ratings >= threshold)
#     user_to_books_relevant = {}
#     for (user, book), rating in test_dict.items():
#         if rating >= relevance_threshold:
#             if user not in user_to_books_relevant:
#                 user_to_books_relevant[user] = set()
#             user_to_books_relevant[user].add(book)
    
#     print(f"  Users with relevant items: {len(user_to_books_relevant)}")
    
#     # Sample users
#     users_to_evaluate = list(user_to_books_relevant.keys())[:max_users]
#     print(f"  Evaluating {len(users_to_evaluate)} users...")
    
#     metrics_list = []
    
#     for idx, user in enumerate(users_to_evaluate):
#         if (idx + 1) % 10 == 0:
#             print(f"    {idx+1}/{len(users_to_evaluate)} users...")
        
#         ground_truth = user_to_books_relevant[user]
        
#         # Get user's training books
#         user_idx = user_to_idx[user]
#         user_ratings = rating_matrix[user_idx].toarray().flatten()
#         rated_indices = np.where(user_ratings > 0)[0]
        
#         # OPTIMIZATION 1: Only predict for popular books (faster)
#         # Get top 100 most popular books as candidates
#         popular_books = np.argsort(book_means)[-100:][::-1]
#         candidate_books = [idx_to_book[i] for i in popular_books if i not in rated_indices]
        
#         # Limit candidates to max 50 books
#         candidate_books = candidate_books[:50]
        
#         # OPTIMIZATION 2: Vectorized prediction for batch
#         predictions = []
#         for book in candidate_books:
#             pred_rating = predict_rating(user, book, k=k_neighbors)
#             predictions.append((book, pred_rating))
        
#         # Sort and get top-K
#         predictions_sorted = sorted(predictions, key=lambda x: x[1], reverse=True)
#         top_k_books = [book for book, score in predictions_sorted[:k_items]]
        
#         # Calculate metrics
#         num_relevant = len(set(top_k_books).intersection(ground_truth))
        
#         precision = num_relevant / k_items if k_items > 0 else 0
#         recall = num_relevant / len(ground_truth) if len(ground_truth) > 0 else 0
#         f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
        
#         metrics_list.append((precision, recall, f1))
    
#     # Calculate averages
#     return np.mean([m[0] for m in metrics_list]), \
#            np.mean([m[1] for m in metrics_list]), \
#            np.mean([m[2] for m in metrics_list])


# # FAST EVALUATION
# print("\nEvaluating ranking metrics")

# RELEVANCE_THRESHOLD = 4.0
# k_items = 10
# k_neighbors = 25

# avg_precision, avg_recall, avg_f1 = evaluate_ranking_metrics_fast(
#     user_book_to_rating_test,
#     k_neighbors=k_neighbors,
#     k_items=k_items,
#     relevance_threshold=RELEVANCE_THRESHOLD,
#     max_users=100
# )

# print(f"Results for K={k_neighbors}, Top-{k_items} recommendations:")
# print(f"Precision@{k_items}: {avg_precision:.4f}")
# print(f"Recall@{k_items}: {avg_recall:.4f}")
# print(f"F1@{k_items}: {avg_f1:.4f}")


Evaluating ranking metrics
  Users with relevant items: 19691
  Evaluating 100 users...
    10/100 users...
    20/100 users...
    30/100 users...
    40/100 users...
    50/100 users...
    60/100 users...
    70/100 users...
    80/100 users...
    90/100 users...
    100/100 users...
Results for K=25, Top-10 recommendations:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000


In [30]:
# k_items = 10
# candidate_k_neighbors = [10, 20, 30, 40, 50]

# results = {}

# print(f"\nTesting different K values for Top-{k_items} recommendations:")
# for k in candidate_k_neighbors:
#     print(f"\n  Testing K={k}...")
    
#     avg_precision, avg_recall, avg_f1 = evaluate_ranking_metrics(
#         user_book_to_rating_test,
#         k_neighbors=k,
#         k_items=k_items,
#         relevance_threshold=RELEVANCE_THRESHOLD,
#         max_users=100
#     )
    
#     results[k] = {
#         f'Precision@{k_items}': avg_precision,
#         f'Recall@{k_items}': avg_recall,
#         f'F1@{k_items}': avg_f1
#     }
    
#     print(f"    Precision@{k_items}: {avg_precision:.4f}")
#     print(f"    Recall@{k_items}: {avg_recall:.4f}")
#     print(f"    F1@{k_items}: {avg_f1:.4f}")



Testing different K values for Top-10 recommendations:

  Testing K=10...
  Users with relevant items: 19691
  Sampling 100 users for evaluation...


KeyboardInterrupt: 

In [ ]:
# # Best K for F1 score
# best_k = max(results.keys(), key=lambda k: results[k][f'F1@{k_items}'])

# print(f"\nBest K value (by F1 score): {best_k}")
# print(f"  Precision@{k_items}: {results[best_k][f'Precision@{k_items}']:.4f}")
# print(f"  Recall@{k_items}: {results[best_k][f'Recall@{k_items}']:.4f}")
# print(f"  F1@{k_items}: {results[best_k][f'F1@{k_items}']:.4f}")

# print("\nRating Prediction Accuracy:")
# best_rating_k = min(rating_results.keys(), key=lambda k: rating_results[k]['RMSE'])
# print(f"Best K for RMSE: {best_rating_k}")
# print(f"  RMSE: {rating_results[best_rating_k]['RMSE']:.4f}")
# print(f"  MAE: {rating_results[best_rating_k]['MAE']:.4f}")

# # Save results
# print("\n[STEP 9] Saving evaluation results...")

# eval_results = {
#     'ranking_metrics': results,
#     'rating_metrics': rating_results,
#     'best_k_ranking': best_k,
#     'best_k_rating': best_rating_k,
#     'test_size': len(test_data_filtered),
#     'relevance_threshold': RELEVANCE_THRESHOLD,
#     'evaluation_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# }

# results_path = r"D:\GoodreadsData\mappings\evaluation_results.pkl"
# with open(results_path, 'wb') as f:
#     pickle.dump(eval_results, f)

# print(f"✓ Results saved to: {results_path}")

# # Create a summary DataFrame
# print("RESULTS TABLE")

# ranking_df = pd.DataFrame(results).T
# ranking_df.index.name = 'K_neighbors'
# print("\nRanking Metrics:")
# print(ranking_df.round(4))

# rating_df = pd.DataFrame(rating_results).T
# rating_df.index.name = 'K_neighbors'
# print("\nRating Prediction Metrics:")
# print(rating_df.round(4))

# print("\n" + "="*70)
# print("✓ EVALUATION COMPLETE!")
# print("="*70)


============================

# everything below is not being used----------------

In [12]:
print(f"Current size: {len(data):,} interactions")
# sampling it down more

Current size: 106,826,044 interactions


In [13]:
# Sample to 1 million interactions
target_size = 1_000_000

if len(data) > target_size:
    print(f"\nSampling to {target_size:,} interactions")
    data = data.sample(n=target_size, random_state=42)
    
    # Re-filter to ensure quality after sampling
    
    # Keep books with at least 20 interactions in sampled data
    book_counts = data['book_id'].value_counts()
    valid_books = book_counts[book_counts >= 20].index
    data = data[data['book_id'].isin(valid_books)].copy()
    
    # Keep users with at least 5 interactions in sampled data
    user_counts = data['user_id'].value_counts()
    valid_users = user_counts[user_counts >= 5].index
    data = data[data['user_id'].isin(valid_users)].copy()


Sampling to 1,000,000 interactions


In [16]:
print("FINAL SAMPLED DATASET:")
print(f"Interactions: {len(data):,}")
print(f"Books: {data['book_id'].nunique():,}")
print(f"Users: {data['user_id'].nunique():,}")
print(f"Sparsity: {100 * (1 - len(data) / (data['user_id'].nunique() * data['book_id'].nunique())):.2f}%")
print(f"Avg interactions/book: {len(data) / data['book_id'].nunique():.1f}")
print(f"Avg interactions/user: {len(data) / data['user_id'].nunique():.1f}")

FINAL SAMPLED DATASET:
Interactions: 144,343
Books: 7,208
Users: 20,780
Sparsity: 99.90%
Avg interactions/book: 20.0
Avg interactions/user: 6.9


In [17]:
print("\nSaving sampled dataset...")
folder = r"D:\GoodreadsData_NEW"
os.makedirs(folder, exist_ok=True)
data.to_csv(os.path.join(folder, 'sampled_1M_data.csv'), index=False)
print("Saved to: D:\\GoodreadsData_NEW\\sampled_1M_data.csv")


Saving sampled dataset...
Saved to: D:\GoodreadsData_NEW\sampled_1M_data.csv


In [20]:
# Create index mappings
users = sorted(data['user_id'].unique())
books = sorted(data['book_id'].unique())

user_to_idx = {user: idx for idx, user in enumerate(users)}
book_to_idx = {book: idx for idx, book in enumerate(books)}
idx_to_user = {idx: user for user, idx in user_to_idx.items()}
idx_to_book = {idx: book for book, idx in book_to_idx.items()}

print(f"Users: {len(users):,}")
print(f"Books: {len(books):,}")

Users: 20,780
Books: 7,208


In [22]:
row_indices = [user_to_idx[uid] for uid in data['user_id']]
col_indices = [book_to_idx[bid] for bid in data['book_id']]
ratings = data['rating'].values

rating_matrix = csr_matrix(
    (ratings, (row_indices, col_indices)),
    shape=(len(users), len(books)),
    dtype=np.float32
)

print(f"Matrix shape: {rating_matrix.shape}")
print(f"Non-zero entries: {rating_matrix.nnz:,}")
print(f"Memory: ~{rating_matrix.data.nbytes / (1024**3):.2f} GB")

print("Sparse matrix created!")

Matrix shape: (20780, 7208)
Non-zero entries: 144,343
Memory: ~0.00 GB
Sparse matrix created!


In [23]:
# Center Ratings

# Compute user means
user_rating_counts = np.diff(rating_matrix.indptr)
user_rating_sums = np.array(rating_matrix.sum(axis=1)).flatten()

# Avoid division by zero
user_means = np.divide(
    user_rating_sums, 
    user_rating_counts, 
    out=np.zeros_like(user_rating_sums), 
    where=user_rating_counts > 0
)

print(f"Mean user rating: {user_means[user_means > 0].mean():.3f}")

# Center ratings (subtract user means)
rating_matrix_centered = rating_matrix.copy().astype(np.float32)

for user_idx in range(rating_matrix_centered.shape[0]):
    start = rating_matrix_centered.indptr[user_idx]
    end = rating_matrix_centered.indptr[user_idx + 1]
    
    if end > start and user_means[user_idx] > 0:
        rating_matrix_centered.data[start:end] -= user_means[user_idx]

# Clean any NaN values
nan_count = np.isnan(rating_matrix_centered.data).sum()
if nan_count > 0:
    print(f"Cleaning {nan_count} NaN values...")
    rating_matrix_centered.data = np.nan_to_num(rating_matrix_centered.data, nan=0.0)

rating_matrix_centered.eliminate_zeros()

print(f"Ratings centered!")
print(f"Non-zero entries after centering: {rating_matrix_centered.nnz:,}")

Mean user rating: 2.109
Ratings centered!
Non-zero entries after centering: 126,435


In [29]:
# Check rating distribution
print("Rating distribution:")
print(data['rating'].value_counts().sort_index())
print(f"Zero ratings: {(data['rating'] == 0).sum()}")
print(f"Non-zero ratings: {(data['rating'] > 0).sum()}")

Rating distribution:
rating
0.0    78557
1.0     1336
2.0     3837
3.0    14723
4.0    24423
5.0    21467
Name: count, dtype: int64
Zero ratings: 78557
Non-zero ratings: 65786


In [24]:
# STEP 7: Compute Item-Item Similarity

start_time = datetime.now()

# Compute adjusted cosine similarity
item_similarity_sparse = cosine_similarity(
    rating_matrix_centered.T, 
    dense_output=False
)

elapsed = datetime.now() - start_time
print(f"Similarity computed in {elapsed}")
print(f"Shape: {item_similarity_sparse.shape}")
print(f"Non-zero similarities: {item_similarity_sparse.nnz:,}")
print(f"Memory: ~{item_similarity_sparse.data.nbytes / (1024**3):.2f} GB")

Similarity computed in 0:00:00.053827
Shape: (7208, 7208)
Non-zero similarities: 880,324
Memory: ~0.00 GB


In [25]:
# Filter by Minimum Common Users

# Compute co-occurrence matrix
print("  Computing co-occurrence...")
rating_binary = rating_matrix.copy()
rating_binary.data = np.ones_like(rating_binary.data)
cooccurrence = (rating_binary.T @ rating_binary).toarray()

# Convert to dense for filtering
item_similarity = item_similarity_sparse.toarray()

# Apply minimum common users filter
min_common = 5
print(f"  Applying filter (min_common={min_common})...")
item_similarity[cooccurrence < min_common] = 0
np.fill_diagonal(item_similarity, 1.0)

# Convert back to sparse
item_similarity_sparse = csr_matrix(item_similarity)
del item_similarity  # Free memory

print(f"Filtered!")
print(f"Non-zero similarities: {item_similarity_sparse.nnz:,}")

  Computing co-occurrence...
  Applying filter (min_common=5)...
Filtered!
Non-zero similarities: 7,232


In [26]:
# STEP 9: Compute Book Means

book_rating_counts = np.diff(rating_matrix.tocsc().indptr)
book_rating_sums = np.array(rating_matrix.sum(axis=0)).flatten()
book_means = np.divide(
    book_rating_sums, 
    book_rating_counts,
    out=np.zeros_like(book_rating_sums),
    where=book_rating_counts > 0
)

print(f"Mean book rating: {book_means[book_means > 0].mean():.3f}")
print("Book means computed!")

Mean book rating: 1.702
Book means computed!


In [27]:
# Save the Model

folder = r"D:\GoodreadsData_NEW"
os.makedirs(folder, exist_ok=True)

save_data = {
    'rating_matrix': rating_matrix,
    'item_similarity': item_similarity_sparse,
    'user_means': user_means,
    'book_means': book_means,
    'user_to_idx': user_to_idx,
    'book_to_idx': book_to_idx,
    'idx_to_user': idx_to_user,
    'idx_to_book': idx_to_book,
    'min_common': min_common
}

model_file = os.path.join(folder, 'sparse_itemcf_model.pkl')
print(f"Saving to: {model_file}")

with open(model_file, 'wb') as f:
    pickle.dump(save_data, f, protocol=pickle.HIGHEST_PROTOCOL)

file_size = os.path.getsize(model_file) / (1024**2)
print(f"Model saved! Size: {file_size:.1f} MB")

Saving to: D:\GoodreadsData_NEW\sparse_itemcf_model.pkl
Model saved! Size: 2.2 MB


In [28]:
# STEP 11: Define Prediction Function & Test

def predict_rating(user_id, book_id, k=25):
    """Predict rating for a user-book pair"""
    if user_id not in user_to_idx or book_id not in book_to_idx:
        return book_means.mean() if len(book_means) > 0 else 3.0
    
    user_idx = user_to_idx[user_id]
    book_idx = book_to_idx[book_id]
    
    # Get user's rated books
    user_ratings = rating_matrix[user_idx].toarray().flatten()
    rated_mask = user_ratings > 0
    
    if not rated_mask.any():
        return book_means[book_idx]
    
    # Get similarities to target book
    similarities = item_similarity_sparse[book_idx].toarray().flatten()
    similarities = similarities * rated_mask
    
    # Get top-k neighbors
    neighbor_indices = np.argsort(np.abs(similarities))[-k:]
    neighbor_indices = neighbor_indices[similarities[neighbor_indices] != 0]
    
    if len(neighbor_indices) == 0:
        return book_means[book_idx]
    
    # Weighted average prediction
    numerator = 0.0
    denominator = 0.0
    
    for idx in neighbor_indices:
        sim = similarities[idx]
        rating = user_ratings[idx]
        baseline = book_means[idx]
        numerator += sim * (rating - baseline)
        denominator += abs(sim)
    
    if denominator == 0:
        return book_means[book_idx]
    
    return np.clip(book_means[book_idx] + numerator / denominator, 0, 5)


def get_recommendations(user_id, n=10, k=25):
    """Get top N recommendations for a user"""
    if user_id not in user_to_idx:
        # Cold start: return most popular books
        top_books = np.argsort(book_means)[-n:][::-1]
        return [(idx_to_book[idx], book_means[idx]) for idx in top_books]
    
    user_idx = user_to_idx[user_id]
    
    # Get rated books
    user_ratings = rating_matrix[user_idx].toarray().flatten()
    rated_books = set(np.where(user_ratings > 0)[0])
    
    # Get unrated books
    all_books = set(range(len(book_means)))
    unrated_books = list(all_books - rated_books)
    
    # Predict ratings
    predictions = []
    for book_idx in unrated_books:
        book_id = idx_to_book[book_idx]
        pred_rating = predict_rating(user_id, book_id, k=k)
        predictions.append((book_id, pred_rating))
    
    # Sort and return top N
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:n]


# Test with a random user
test_user = list(user_to_idx.keys())[0]
print(f"\nTest User: {test_user}")

# Get user's actual ratings
user_idx = user_to_idx[test_user]
user_ratings = rating_matrix[user_idx].toarray().flatten()
rated_indices = np.where(user_ratings > 0)[0]
rated_books = [(idx_to_book[idx], user_ratings[idx]) for idx in rated_indices]

print(f"\nUser has rated {len(rated_books)} books")
print("Sample ratings:")
for book_id, rating in rated_books[:5]:
    print(f"  Book {book_id}: {rating:.1f} stars")

# Test prediction on a rated book
if len(rated_books) > 0:
    test_book = rated_books[0][0]
    actual_rating = rated_books[0][1]
    predicted = predict_rating(test_user, test_book, k=25)
    print(f"\nPrediction test:")
    print(f"  Book {test_book}")
    print(f"  Actual rating: {actual_rating:.1f}")
    print(f"  Predicted rating: {predicted:.2f}")

# Get recommendations
print(f"\nTop 10 Recommendations for User {test_user}:")
recs = get_recommendations(test_user, n=10, k=25)
for i, (book_id, pred_rating) in enumerate(recs, 1):
    print(f"  {i:2d}. Book {book_id:6d} - Predicted: {pred_rating:.2f} stars")

print("Model Summary:")
print(f"Dataset: {len(data):,} interactions")
print(f"Users: {len(users):,}")
print(f"Books: {len(books):,}")
print(f"Model size: {file_size:.1f} MB")
print(f"Prediction function: predict_rating(user_id, book_id, k=25)")
print(f"Recommendation function: get_recommendations(user_id, n=10, k=25)")


Test User: 5

User has rated 2 books
Sample ratings:
  Book 6955: 4.0 stars
  Book 7057: 5.0 stars

Prediction test:
  Book 6955
  Actual rating: 4.0
  Predicted rating: 4.00

Top 10 Recommendations for User 5:
   1. Book  23153 - Predicted: 5.00 stars
   2. Book  23368 - Predicted: 5.00 stars
   3. Book  59387 - Predicted: 5.00 stars
   4. Book  15362 - Predicted: 4.86 stars
   5. Book  19410 - Predicted: 4.86 stars
   6. Book    270 - Predicted: 4.75 stars
   7. Book  20658 - Predicted: 4.75 stars
   8. Book    665 - Predicted: 4.50 stars
   9. Book  14479 - Predicted: 4.50 stars
  10. Book  19488 - Predicted: 4.50 stars
Model Summary:
Dataset: 144,343 interactions
Users: 20,780
Books: 7,208
Model size: 2.2 MB
Prediction function: predict_rating(user_id, book_id, k=25)
Recommendation function: get_recommendations(user_id, n=10, k=25)


==========================================================================

In [15]:
# print("Filtering books with minimum 20 interactions...")
# data = original_data.groupby('book_id').filter(lambda x: len(x) >= 20)

Filtering books with minimum 20 interactions...


In [17]:
# print(f"Filtered dataset:")
# print(f"Total interactions: {len(data):,}")
# print(f"Unique users: {data['user_id'].nunique():,}")
# print(f"Unique books: {data['book_id'].nunique():,}")
# print(f"Sparsity: {100 * (1 - len(data) / (data['user_id'].nunique() * data['book_id'].nunique())):.2f}%")


Filtered dataset:
Total interactions: 115,346,986
Unique users: 254,194
Unique books: 578,590
Sparsity: 99.92%


In [19]:
#creating sparse rating matrix

# Create index mappings
users = sorted(data['user_id'].unique())
books = sorted(data['book_id'].unique())

user_to_idx = {user: idx for idx, user in enumerate(users)}
book_to_idx = {book: idx for idx, book in enumerate(books)}
idx_to_user = {idx: user for user, idx in user_to_idx.items()}
idx_to_book = {idx: book for book, idx in book_to_idx.items()}

print(f"Number of users: {len(users):,}")
print(f"Number of books: {len(books):,}")

# Create sparse matrix
row_indices = [user_to_idx[uid] for uid in data['user_id']]
col_indices = [book_to_idx[bid] for bid in data['book_id']]
ratings = data['rating'].values

rating_matrix = csr_matrix(
    (ratings, (row_indices, col_indices)),
    shape=(len(users), len(books)),
    dtype=np.float32
)

print(f"Rating matrix shape: {rating_matrix.shape}")
print(f"Non-zero entries: {rating_matrix.nnz:,}")
print(f"Matrix memory: ~{rating_matrix.data.nbytes / (1024**3):.2f} GB")

Number of users: 254,194
Number of books: 578,590
Rating matrix shape: (254194, 578590)
Non-zero entries: 115,346,986
Matrix memory: ~0.43 GB


In [20]:
# Computing user means and centering ratings

# Compute user means
user_rating_counts = np.diff(rating_matrix.indptr)
user_rating_sums = np.array(rating_matrix.sum(axis=1)).flatten()
user_means = np.divide(user_rating_sums, user_rating_counts, 
                      out=np.zeros_like(user_rating_sums), 
                      where=user_rating_counts!=0)

print(f"Mean user rating: {user_means.mean():.3f}")
print(f"Starting to center ratings...")

# Center ratings
rating_matrix_centered = rating_matrix.copy().astype(np.float32)

for user_idx in range(rating_matrix_centered.shape[0]):
    start = rating_matrix_centered.indptr[user_idx]
    end = rating_matrix_centered.indptr[user_idx + 1]
    
    if end > start:
        rating_matrix_centered.data[start:end] -= user_means[user_idx]
    
    # Progress indicator every 50K users
    if (user_idx + 1) % 50000 == 0:
        print(f"  Centered {user_idx + 1:,} / {len(users):,} users ({100*(user_idx+1)/len(users):.1f}%)...")

print("Ratings centered successfully ---> DONE")

Mean user rating: nan
Starting to center ratings...
  Centered 50,000 / 254,194 users (19.7%)...
  Centered 100,000 / 254,194 users (39.3%)...
  Centered 150,000 / 254,194 users (59.0%)...
  Centered 200,000 / 254,194 users (78.7%)...
  Centered 250,000 / 254,194 users (98.4%)...
Ratings centered successfully ---> DONE


In [21]:
#SAVING CHECKPOINT!

folder = r"D:\GoodreadsData_SPARSE"
os.makedirs(folder, exist_ok=True)

checkpoint_data = {
    'rating_matrix': rating_matrix,
    'rating_matrix_centered': rating_matrix_centered,
    'user_means': user_means,
    'user_to_idx': user_to_idx,
    'book_to_idx': book_to_idx,
    'idx_to_user': idx_to_user,
    'idx_to_book': idx_to_book,
    'users': users,
    'books': books
}

checkpoint_file = os.path.join(folder, 'checkpoint_before_similarity.pkl')

print(f"Saving checkpoint to: {checkpoint_file}")
with open(checkpoint_file, 'wb') as f:
    pickle.dump(checkpoint_data, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Checkpoint saved successfully!")
print(f"File size: {os.path.getsize(checkpoint_file) / (1024**3):.2f} GB")

Saving checkpoint to: D:\GoodreadsData_SPARSE\checkpoint_before_similarity.pkl
Checkpoint saved successfully!
File size: 1.75 GB


In [ ]:
# print("Loading checkpoint...")

# folder = r"D:\GoodreadsData"
# checkpoint_file = os.path.join(folder, 'checkpoint_before_similarity.pkl')

# with open(checkpoint_file, 'rb') as f:
#     checkpoint_data = pickle.load(f)

# # Restore all variables
# rating_matrix = checkpoint_data['rating_matrix']
# rating_matrix_centered = checkpoint_data['rating_matrix_centered']
# user_means = checkpoint_data['user_means']
# user_to_idx = checkpoint_data['user_to_idx']
# book_to_idx = checkpoint_data['book_to_idx']
# idx_to_user = checkpoint_data['idx_to_user']
# idx_to_book = checkpoint_data['idx_to_book']
# users = checkpoint_data['users']
# books = checkpoint_data['books']

# print("✓ Checkpoint loaded successfully!")
# print(f"  Rating matrix: {rating_matrix.shape}")
# print(f"  Users: {len(users):,}")
# print(f"  Books: {len(books):,}")
# print("\nYou can now continue with Step 4!")

In [23]:
# Checking for NaN values
print(f"NaN values in rating_matrix_centered: {np.isnan(rating_matrix_centered.data).sum():,}")

if np.isnan(rating_matrix_centered.data).any():
    print("Cleaning NaN values (replacing with 0)...")
    rating_matrix_centered.data = np.nan_to_num(rating_matrix_centered.data, nan=0.0)
    print("NaN values cleaned!")
else:
    print("No NaN values found!")

print(f"NaN count: {np.isnan(rating_matrix_centered.data).sum()}")

NaN values in rating_matrix_centered: 220
Cleaning NaN values (replacing with 0)...
NaN values cleaned!
NaN count: 0


In [24]:
rating_matrix_centered.eliminate_zeros()
print(f"Non-zero entries: {rating_matrix_centered.nnz:,}")

Non-zero entries: 114,903,391


In [25]:
#Computing adjusted cosine similarity

start_time = datetime.now()

try:
    item_similarity_sparse = cosine_similarity(
        rating_matrix_centered.T, 
        dense_output=False
    )
    
    elapsed = datetime.now() - start_time
    print(f"DONEE! Similarity computed in {elapsed}")
    print(f"Shape: {item_similarity_sparse.shape}")
    print(f"Non-zero similarities: {item_similarity_sparse.nnz:,}")
    print(f"Memory: ~{item_similarity_sparse.data.nbytes / (1024**3):.2f} GB")
    
    # Save immediately
    folder = r"D:\GoodreadsData_SPARSE"
    with open(os.path.join(folder, 'similarity_matrix_raw.pkl'), 'wb') as f:
        pickle.dump(item_similarity_sparse, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("Similarity matrix saved")
    
except MemoryError:
    print("OUT OF MEMORY!")
    
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

OUT OF MEMORY!
Suggestion: Increase threshold to 30 and restart


In [29]:
# SAMPLING STRATEGY: Hybrid (Popular Books + Active Users)

# Keep books with at least 50 interactions (stricter filter)
data = data.groupby('book_id').filter(lambda x: len(x) >= 50)
print(f"After filtering: {len(data):,} interactions, {data['book_id'].nunique():,} books")

# Keep users with at least 10 interactions (active users)
data = data.groupby('user_id').filter(lambda x: len(x) >= 10)
print(f"After filtering: {len(data):,} interactions, {data['user_id'].nunique():,} users")

# If still too large, sample to target size
target_max = 1_000_000  # 1M interactions
if len(data) > target_max:
    print(f"Dataset still large ({len(data):,}), sampling to {target_max:,}...")
    sample_fraction = target_max / len(data)
    data = data.sample(frac=sample_fraction, random_state=42)
    print(f"Sampled to: {len(data):,} interactions")

print("FINAL DATASET:")
print(f"Total interactions: {len(data):,}")
print(f"Unique books: {data['movie_id'].nunique():,}")
print(f"Unique users: {data['user_id'].nunique():,}")
print(f"Sparsity: {100 * (1 - len(data) / (data['user_id'].nunique() * data['movie_id'].nunique())):.2f}%")
print(f"Avg interactions per book: {len(data) / data['movie_id'].nunique():.1f}")
print(f"Avg interactions per user: {len(data) / data['user_id'].nunique():.1f}")

After filtering: 106,826,056 interactions, 306,020 books


MemoryError: Unable to allocate 2.39 GiB for an array with shape (3, 106825306) and data type float64

-----------------------------------------

In [6]:
# avoids calculating similarity for millions of books
data = original_data.groupby('movie_id').filter(lambda x: len(x) >= 30)

In [8]:
data.head()

,user_id,movie_id,is_read,rating,is_reviewed
0,0,948,1.0,5.0,0.0
1,0,947,1.0,5.0,1.0
2,0,946,1.0,5.0,0.0
3,0,945,1.0,5.0,0.0
4,0,944,1.0,5.0,0.0


In [9]:
len(data.movie_id.unique())

444259

In [10]:
len(data)

112120510

In [12]:
print("Before filtering:", len(original_data))
print("After filtering:", len(data))

Before filtering: 123731803
After filtering: 112120510


In [13]:
print("Users:", data.user_id.nunique())
print("Books:", data.movie_id.nunique())
print("Interactions:", len(data))

Users: 254191
Books: 444259
Interactions: 112120510


In [14]:
user_to_movie = data.groupby('user_id')['movie_id'].apply(list).to_dict()
movie_to_user = data.groupby('movie_id')['user_id'].apply(list).to_dict()

In [15]:
user_to_movie[0][0]

948

In [16]:
user_to_ratings = data.groupby('user_id')['rating'].apply(list).to_dict()

In [23]:
user_to_ratings[0][0]

5.0

In [29]:
import pickle

# Replace this path with your folder on D: drive
folder = r"D:\GoodreadsData"

# Make sure the folder exists
import os
os.makedirs(folder, exist_ok=True)

# Save dictionaries
with open(os.path.join(folder, "user_to_movie.pkl"), "wb") as f:
    pickle.dump(user_to_movie, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(folder, "user_to_ratings.pkl"), "wb") as f:
    pickle.dump(user_to_ratings, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(folder, "movie_to_user.pkl"), "wb") as f:
    pickle.dump(movie_to_user, f, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
user_to_movie_rating = (
    data.groupby('user_id')
        .apply(lambda x: list(zip(x['movie_id'], x['rating'])))
        .to_dict()
)

In [38]:
user_to_movie_rating[0]

[(948, 5.0),
 (947, 5.0),
 (946, 5.0),
 (945, 5.0),
 (944, 5.0),
 (943, 5.0),
 (942, 5.0),
 (941, 5.0),
 (940, 5.0),
 (939, 5.0),
 (938, 5.0),
 (937, 4.0),
 (936, 4.0),
 (935, 4.0),
 (934, 5.0),
 (933, 4.0),
 (932, 4.0),
 (931, 5.0),
 (930, 2.0),
 (929, 4.0),
 (928, 4.0),
 (927, 5.0),
 (924, 5.0),
 (923, 5.0),
 (922, 5.0),
 (921, 4.0),
 (920, 5.0),
 (919, 5.0),
 (918, 5.0),
 (917, 3.0),
 (916, 5.0),
 (915, 5.0),
 (914, 4.0),
 (912, 5.0),
 (910, 0.0),
 (909, 4.0),
 (908, 4.0),
 (907, 0.0),
 (906, 0.0),
 (905, 4.0),
 (904, 0.0),
 (903, 5.0),
 (901, 4.0),
 (900, 4.0),
 (899, 4.0),
 (898, 3.0),
 (897, 5.0),
 (896, 0.0),
 (895, 5.0),
 (893, 4.0),
 (892, 5.0),
 (891, 4.0),
 (890, 5.0),
 (889, 5.0),
 (888, 4.0),
 (887, 0.0),
 (886, 3.0),
 (885, 0.0),
 (884, 0.0),
 (883, 4.0),
 (882, 5.0),
 (881, 0.0),
 (880, 5.0),
 (879, 5.0),
 (878, 4.0),
 (877, 5.0),
 (876, 4.0),
 (875, 4.0),
 (874, 3.0),
 (873, 4.0),
 (872, 3.0),
 (871, 2.0),
 (870, 3.0),
 (869, 3.0),
 (868, 0.0),
 (867, 3.0),
 (866, 4.0),

In [32]:
with open(os.path.join(folder, "user_to_movie_rating.pkl"), "wb") as f:
    pickle.dump(user_to_movie_rating, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(r"D:\GoodreadsData\user_to_movie.pkl", "rb") as f:
#     user_to_movie = pickle.load(f)

# with open(r"D:\GoodreadsData\user_to_ratings.pkl", "rb") as f:
#     user_to_ratings = pickle.load(f)

# with open(r"D:\GoodreadsData\movie_to_user.pkl", "rb") as f:
#     movie_to_user = pickle.load(f)

# with open(r"D:\GoodreadsData\user_to_movie_rating.pkl", "rb") as f:
#     user_to_movie_rating = pickle.load(f)

In [34]:
user_mean = {}
for user, movie_ratings in user_to_movie_rating.items():
    total = 0
    count = 0
    for movie, rating in movie_ratings:
        total += rating
        count += 1
    user_mean[user] = total / count

In [36]:
user_mean[0]

2.1421911421911424

In [40]:
movie_mean = data.groupby('movie_id')['rating'].mean().to_dict()

In [41]:
movie_mean[0]

0.029017857142857144

In [42]:
import pickle

# Save user_mean
with open("user_mean.pkl", "wb") as f:
    pickle.dump(user_mean, f, protocol=pickle.HIGHEST_PROTOCOL)

# Save movie_mean
with open("movie_mean.pkl", "wb") as f:
    pickle.dump(movie_mean, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# import pickle

# with open("user_mean.pkl", "rb") as f:
#     user_mean = pickle.load(f)

# with open("movie_mean.pkl", "rb") as f:
#     movie_mean = pickle.load(f)

In [43]:
def adjusted_cosine_similarity(movie_i, movie_j, min_common=5):
    users_i = set(movie_to_user[movie_i])
    users_j = set(movie_to_user[movie_j])
    common_users = users_i.intersection(users_j)

    if len(common_users) < min_common:
        return 0

    numerator = 0
    denominator_i = 0
    denominator_j = 0

    for user in common_users:
        rating_i = user_movie_to_rating[(user, movie_i)]
        rating_j = user_movie_to_rating[(user, movie_j)]
        diff_i = rating_i - user_mean[user]
        diff_j = rating_j - user_mean[user]

        numerator += diff_i * diff_j
        denominator_i += diff_i ** 2
        denominator_j += diff_j ** 2

    return numerator / (math.sqrt(denominator_i) * math.sqrt(denominator_j))


In [44]:
def compute_similarity_matrix(min_common=5):
    # 1. Precompute co-rated pairs (scales well)
    co_rated = defaultdict(set)

    for user, movies in user_to_movie.items():
        for i in range(len(movies)):
            for j in range(i+1, len(movies)):
                a, b = movies[i], movies[j]
                co_rated[a].add(b)
                co_rated[b].add(a)

    # 2. Compute similarity only for those pairs
    movies = list(movie_to_user.keys())
    similarity_matrix = {}

    for idx, movie_i in enumerate(movies):
        similarity_matrix[movie_i] = {}

        for movie_j in co_rated[movie_i]:
            sim = adjusted_cosine_similarity(movie_i, movie_j, min_common)
            similarity_matrix[movie_i][movie_j] = sim

        # Save incrementally to avoid memory blowup
        with open(f'sim_{movie_i}.pkl', 'wb') as f:
            pickle.dump(similarity_matrix[movie_i], f)

        if idx % 200 == 0:
            print(f"{idx} movies processed...")

    return similarity_matrix


In [45]:
import pickle
import os

folder = r"D:\GoodreadsData"

similarity_matrix = {}

for movie in movie_to_user.keys():
    filepath = os.path.join(folder, f"sim_{movie}.pkl")
    
    try:
        with open(filepath, "rb") as f:
            similarity_matrix[movie] = pickle.load(f)
    except FileNotFoundError:
        similarity_matrix[movie] = {}

In [49]:
similarity_matrix

{0: {},
 1: {},
 2: {},
 3: {},
 5: {},
 6: {},
 7: {},
 8: {},
 9: {},
 10: {},
 11: {},
 12: {},
 13: {},
 14: {},
 15: {},
 16: {},
 17: {},
 18: {},
 19: {},
 20: {},
 21: {},
 22: {},
 23: {},
 25: {},
 26: {},
 27: {},
 28: {},
 29: {},
 30: {},
 31: {},
 32: {},
 33: {},
 34: {},
 35: {},
 36: {},
 37: {},
 38: {},
 39: {},
 40: {},
 41: {},
 42: {},
 43: {},
 44: {},
 46: {},
 47: {},
 48: {},
 49: {},
 50: {},
 51: {},
 52: {},
 53: {},
 55: {},
 56: {},
 57: {},
 58: {},
 59: {},
 60: {},
 61: {},
 62: {},
 63: {},
 64: {},
 65: {},
 66: {},
 67: {},
 68: {},
 69: {},
 70: {},
 71: {},
 72: {},
 74: {},
 75: {},
 76: {},
 77: {},
 78: {},
 79: {},
 80: {},
 81: {},
 82: {},
 83: {},
 84: {},
 85: {},
 86: {},
 87: {},
 88: {},
 89: {},
 90: {},
 92: {},
 93: {},
 95: {},
 96: {},
 97: {},
 98: {},
 99: {},
 100: {},
 101: {},
 103: {},
 104: {},
 105: {},
 106: {},
 108: {},
 109: {},
 110: {},
 111: {},
 112: {},
 113: {},
 115: {},
 116: {},
 117: {},
 118: {},
 119: {},
 1

In [46]:
def predict_rating(user, target_movie, k=25):
    rated_movies = user_to_movie[user]
    similarities = []

    for movie in rated_movies:
        if (target_movie in similarity_matrix) and (movie in similarity_matrix[target_movie]):
            similarity = similarity_matrix[target_movie][movie]
        elif (movie in similarity_matrix) and (target_movie in similarity_matrix[movie]):
            similarity = similarity_matrix[movie][target_movie]
        else:
            similarity = 0

        similarities.append((movie, similarity))

    similarities.sort(key=lambda x: abs(x[1]), reverse=True)
    neighbors = similarities[:k]

    numerator = 0
    denominator = 0
    for movie, sim in neighbors:
        rating = user_movie_to_rating[(user, movie)]
        base = movie_mean[movie]
        numerator += sim * (rating - base)
        denominator += abs(sim)

    return movie_mean[target_movie] + numerator / (denominator + 1e-9)


In [47]:
target_user = 948   # example from your CSV

all_movies = set(movie_to_user.keys())
rated_movies = set(user_to_movie[target_user])
candidate_movies = all_movies - rated_movies

predictions = {}

for idx, movie in enumerate(candidate_movies):
    if idx % 200 == 0:
        print(f"{idx} movies processed...")

    predictions[movie] = predict_rating(target_user, movie)


0 movies processed...


NameError: name 'user_movie_to_rating' is not defined

In [ ]:
top_5 = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:5]

print("\nTop 5 Recommendations:")
for movie, pred in top_5:
    print(f"Book {movie}, Predicted Rating: {round(pred, 3)}")